In [40]:
import pandas as pd
import numpy as np
import math

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold

import platform
from os import listdir
from os.path import isfile, join
from glob import glob
from pathlib import Path
import sys
import os



import matplotlib.pyplot as plt

import SMOTE
import feature_selector
import DE
import CFS

import metrices
import measures

import sys
import traceback
import warnings
warnings.filterwarnings("ignore")

In [41]:
data_source1 = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/data/commit_guru'
if platform.system() == 'Darwin' or platform.system() == 'Linux':
    _dir = data_source1 + '/'
else:
    _dir = data_source1 + '\\'
projects1 = [f for f in listdir(_dir) if isfile(join(_dir, f))]

data_source2 = '/Users/suvodeepmajumder/Documents/AI4SE/git_miner/data'
if platform.system() == 'Darwin' or platform.system() == 'Linux':
    _dir = data_source2 + '/'
else:
    _dir = data_source2 + '\\'
projects2 = [f for f in listdir(_dir) if isfile(join(_dir, f))]

commit_list = []
for project in projects2:
    x = project.split('_commit')[0]
    if x not in commit_list:
        commit_list.append(x)
commit_list = set(commit_list)

guru_list = []
for project in projects1:
    x = project.split('.csv')[0]
    if x not in guru_list:
        guru_list.append(x)
guru_list = set(guru_list)

if (commit_list & guru_list): 
    common_list = commit_list & guru_list

In [42]:
def prepare_data(path,X):
    df = pd.read_csv(path)
    df = df[df['commit_hash'].isin(X)]
    df = df.drop(labels = ['commit_hash', 'author_name', 'author_date_unix_timestamp',
       'author_email', 'author_date', 'commit_message','classification', 'linked', 'contains_bug', 'fixes',
                      'fileschanged','glm_probability', 'rf_probability',
       'repository_id', 'issue_id', 'issue_date', 'issue_type'],axis=1)
    df = df.dropna()
    df = df[['ns', 'nd', 'nf', 'entropy', 'la', 'ld', 'lt', 'ndev', 'age',
           'nuc', 'exp', 'rexp', 'sexp','fix']]
    df = df.astype(np.float64)
    return df

def get_features(df):
    fs = feature_selector.featureSelector()
    df,_feature_nums,features = fs.cfs_bfs(df)
    return df,features

def apply_cfs(df):
    y = df.fix.values
    X = df.drop(labels = ['fix'],axis = 1)
    X = X.values
    selected_cols = CFS.cfs(X,y)
    cols = df.columns[[selected_cols]].tolist()
    cols.append('fix')
    return df[cols],cols
    
def apply_smote(df):
    cols = df.columns
    smt = SMOTE.smote(df)
    df = smt.run()
    df.columns = cols
    return df

def tune_learner(learner, train_X, train_Y, tune_X, tune_Y, goal,target_class=None):
    if not target_class:
        target_class = goal
    clf = learner(train_X, train_Y, tune_X, tune_Y, goal)
    tuner = DE.DE_Tune_ML(clf, clf.get_param(), goal, target_class)
    return tuner.Tune()

In [43]:
class DE_Learners(object):
    def __init__(self, clf, train_X, train_Y, test_X, test_Y, goal):
        """

        :param clf: classifier, SVM, etc...
        :param train_X: training data, independent variables.
        :param train_Y: training labels, dependent variables.
        :param predict_X: testing data, independent variables.
        :param predict_Y: testingd labels, dependent variables.
        :param goal: the objective of your tuning, F, recision,....
        """
        self.train_X = train_X
        self.train_Y = train_Y
        self.test_X = test_X
        self.test_Y = test_Y
        self.goal = goal
        self.param_distribution = self.get_param()
        self.learner = clf
        self.confusion = None
        self.params = None

    def learn(self,F, **kwargs):
        """
        :param F: a dict, holds all scores, can be used during debugging
        :param kwargs: a dict, all the parameters need to set after tuning.
        :return: F, scores.
        """
        self.scores = {self.goal: [0.0]}
        try:    
            self.learner.set_params(**kwargs)
            predict_result = []
            clf = self.learner.fit(self.train_X, self.train_Y)
            predict_result = clf.predict(self.test_X)
            self.abcd = metrices.measures(self.test_Y,predict_result)
            self.scores = self._Abcd(self.abcd,F)
            self.confusion = metrics.classification_report(self.test_Y.values.tolist(), predict_result, digits=2)
            self.params = kwargs
        except Exception as e:
            a = 10
        return self.scores
    
    def _Abcd(self,abcd , F):
        """

        :param predicted: predicted results(labels)
        :param actual: actual results(labels)
        :param F: previously got scores
        :return: updated scores.
        """
        if 'g-score' in self.goal:
            F['g-score'] = [abcd.get_g_score()]
            return F
        elif 'precision' in self.goal:
            F['precision'] = [abcd.get_precision()]
            return F
        elif 'f1' in self.goal:
            F['f1'] = [abcd.calculate_f1_score()]
            return F
        elif 'd2h' in self.goal:
            F['d2h'] = [abcd.calculate_d2h()]
            return F

    def predict(self,test_X):
        return self.learner.predict(test_X)

In [44]:
class SK_LR(DE_Learners):
    def __init__(self, train_x, train_y, predict_x, predict_y, goal):
        clf = LogisticRegression()
        super(SK_LR, self).__init__(clf, train_x, train_y, predict_x, predict_y,goal)

    def get_param(self):
        tunelst = {"penalty": ['l1', 'l2','elasticnet',None],
                   "multi_class": ['ovr', 'multinomial','auto'],
                   "C": [1.0,200.0],
                   "dual": [True, False],
                   "fit_intercept": [True, False],
                   "intercept_scaling": [1.0,100.0],
                   "class_weight": ["balanced", None],
                   "solver": ['newton-cg','lbfgs','liblinear','sag', 'saga'],
                   "warm_start": [True, False],
                   "max_iter": [100,600]}
        return tunelst

In [36]:
df = pd.read_pickle('/Users/suvodeepmajumder/Documents/AI4SE/git_miner/data/BaseRecyclerViewAdapterHelper_commit.pkl')
df1 = df[df['buggy'] == 1]
X1 = df1.commit_number
X2 = df1.parent
X = np.append(X1,X2)

In [37]:
path = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/data/commit_guru/BaseRecyclerViewAdapterHelper.csv'
df = prepare_data(path,X)
#df,features = get_features(df)
df.reset_index(drop=True,inplace=True)
y = df.fix
X = df.drop(labels = ['fix'],axis = 1)

In [54]:
commit_list = ['BaseRecyclerViewAdapterHelper']
final_score = {}
for project in commit_list:
    try:
        df = pd.read_pickle('/Users/suvodeepmajumder/Documents/AI4SE/git_miner/data/' + project + '_commit.pkl')
        df1 = df[df['buggy'] == 1]
        X1 = df1.commit_number
        X2 = df1.parent
        X = np.append(X1,X2)
        path = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/data/commit_guru/' + project + '.csv'
        df = prepare_data(path,X)
        df.reset_index(drop=True,inplace=True)
        y = df.fix
        X = df.drop(labels = ['fix'],axis = 1)
        train_X,test_X,train_y,test_y = train_test_split(X, y, test_size=0.33, random_state=13)
        df = pd.concat([train_X,train_y], axis = 1)
        df.reset_index(drop=True,inplace=True)
        y = df.fix
        X = df.drop(labels = ['fix'],axis = 1)
        kf = StratifiedKFold(n_splits = 5)
        goal = 'f1'
        learner = [SK_LR][0]
        F = {}
        score = {}
        for i in range(2):
            for train_index, tune_index in kf.split(X, y):
                X_train, X_tune = X.iloc[train_index], X.iloc[tune_index]
                y_train, y_tune = y[train_index], y[tune_index]
                _df = pd.concat([X_train,y_train], axis = 1)
                _df_tune = pd.concat([X_tune,y_tune], axis = 1)
                _df = apply_smote(_df)
                _df,selected_cols = apply_cfs(_df)
                y_train = _df.fix
                X_train = _df.drop(labels = ['fix'],axis = 1)
                _df_tune = _df_tune[selected_cols]
                y_tune = _df_tune.fix
                X_tune = _df_tune.drop(labels = ['fix'],axis = 1)
                params, evaluation = tune_learner(learner, X_train, y_train,  X_tune,y_tune, goal)
                clf = learner(X_train, y_train,  test_X,test_y, goal)
                F = clf.learn(F,**params)
                print(F)
                score['f1'].append(F['f1'][0])
                print(score)
                score['precision'].append(F['precision'][0])
                print(score)
                score['recall'].append(F['recall'][0])
                print(score)
                score['g-score'].append(F['g-score'][0])
                print(score)
                score['d2h'].append(F['d2h'][0])
                print(score)
        project_score.append([project,score])
    except Exception as e:
        print(e)
        continue

20190702_12:53:38 ### Now life is:  20
20190702_12:53:38 ### Now life is:  19
20190702_12:53:38 ### Now life is:  18
20190702_12:53:38 ### Now life is:  17
newbestscore {'f1': 0.7777777777777778}:
bestconf {'penalty': 'l2', 'multi_class': 'auto', 'C': 19.98, 'dual': True, 'fit_intercept': True, 'intercept_scaling': 88.609, 'class_weight': 'balanced', 'solver': 'liblinear', 'warm_start': False, 'max_iter': 265} :
20190702_12:53:38 ### Now life is:  17
20190702_12:53:38 ### Now life is:  16
20190702_12:53:38 ### Now life is:  15
20190702_12:53:38 ### Now life is:  14
20190702_12:53:39 ### Now life is:  13
20190702_12:53:39 ### Now life is:  12
20190702_12:53:39 ### Now life is:  11
20190702_12:53:39 ### Now life is:  10
20190702_12:53:39 ### Now life is:  9
20190702_12:53:39 ### Now life is:  8
20190702_12:53:39 ### Now life is:  7
newbestscore {'f1': 0.7960222378827031}:
bestconf {'penalty': 'l2', 'multi_class': 'auto', 'C': 55.062, 'dual': True, 'fit_intercept': False, 'intercept_scali

In [51]:
project_score

[['BaseRecyclerViewAdapterHelper',
  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]]

In [ ]:
plt.boxplot(score)

In [8]:
project_score_df = pd.DataFrame(project_score, columns=['project','score'])

In [9]:
project_score_df.to_csv('results.csv')

In [31]:
cnt = 0
for i in range(project_score_df.shape[0]):
    if round(np.median(np.array(project_score_df.iloc[i,1])),2) >= .75:
        print(project_score_df.iloc[i,0],round(np.median(np.array(project_score_df.iloc[i,1])),2))
        cnt+=1
print(cnt)
    #print(project_score_df.iloc[i,0],round(np.median(np.array(project_score_df.iloc[i,1])),2))

canal 0.77
zsc 0.86
motogp 1.0
docker 0.82
AndroidSwipeLayout 0.83
FizzBuzzEnterpriseEdition 1.0
recyclerview-animators 0.8
Luban 1.0
Fragmentation 0.76
tetris_project 1.0
10
